# Modelo de Apredizaje automatico


### Nerio Morán CI: 23.556.790


# Base de datos Emocional


Unos de los topicos de mayor interes en la interaccion humano robot, es la identificacion de emociones a través de diferentes entradas. Una de las entradas de mayor estudio corresponde a las imagenes de rostros de personas. Aunque existen muchos estudios que demuestran tener una gran precision para determinar una emocion a traves de esta entrada, muhas veces el tipo de aplicacion que se les da a estos clasificadores no permite que se pueda captar de manera adeacuada el rostro de una persona, es por esto que las investigaciones actuales se centran en otros tipos de entradas como: La voz, el ritmo cardiaco, señales inalambricas, señales musculares entre otros.


Para esta investigacion se hizo una base de datos emocional, cuya entrada es la voz. Los datos fueron extraidos de expresiones, reacciones, blogs de videos en youtube. Ciertos criterios fueros tomados para la seleccion de segmento de audio, cada audio tiene un tamaño de 2 a 6 segundos. Fueron transformados a formato wav con una frecuencia de muestreo de 16Khz y una taza de bits de 256.

## Utilidades


In [31]:
import pyaudio
import wave
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import svm, datasets
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
%matplotlib inline
CHUNK = 1024


In [32]:
def play_wav(wav_filename, chunk_size=CHUNK):
    '''
    Play (on the attached system sound device) the WAV file
    named wav_filename.
    '''

    try:
        print ("Trying to play file " + wav_filename)
        wf = wave.open(wav_filename, 'rb')
    except IOError as ioe:
        sys.stderr.write('IOError on file ' + wav_filename + '\n' + \
        str(ioe) + '. Skipping.\n')
        return
    except EOFError as eofe:
        sys.stderr.write('EOFError on file ' + wav_filename + '\n' + \
        str(eofe) + '. Skipping.\n')
        return

    # Instantiate PyAudio.
    p = pyaudio.PyAudio()

    # Open stream.
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
        channels=wf.getnchannels(),
        rate=wf.getframerate(),
                    output=True)

    data = wf.readframes(chunk_size)
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(chunk_size)

    # Stop stream.
    stream.stop_stream()
    stream.close()

    # Close PyAudio.
    p.terminate()


In [33]:
# Ira
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/anger/0-1.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/anger/6-27.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/anger/0-1.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/anger/6-27.wav


In [34]:
# Disgusto
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/disgust/3-12.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/disgust/12-67.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/disgust/3-12.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/disgust/12-67.wav


In [35]:
# Miedo
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/fear/10-54.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/fear/11-66.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/fear/10-54.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/fear/11-66.wav


In [36]:
#Tristeza
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/sadness/5-21.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/sadness/5-24.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/sadness/5-21.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/sadness/5-24.wav


In [37]:
#Sorpresa
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/surprise/3-10.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/surprise/3-11.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/surprise/3-10.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/surprise/3-11.wav


In [38]:
# Felicidad
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-30.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-31.wav",CHUNK);
play_wav("/home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-32.wav",CHUNK);

Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-30.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-31.wav
Trying to play file /home/neriomoran/MachineLearning/AudioProcessing/EmocionesAudio/happiness/7-32.wav


## Los datos consisten en los siguiente:

### Atributos (caracteristicas) del conjunto de datos:
- 1) Taza de cruces por zero (promedio)
- 2) Energia ó Potencia de la señal.  (promedio)
- 3) Entropia de la energia.  (promedio)
- 4) Centroide Espectral.  (promedio)
- 5) Tono.  (promedio)
- 6) Entropia espectral.  (promedio)
- 7) Flujo espectral.  (promedio)
- 8) Caida espectral.  (promedio)
- 9-21) Coeficientes de Mel MFFCs.  (promedio)
- 22-33) Vector cromatico.  (promedio)
- 34)  Desviacion estandar del vector cromatico.  (promedio)
- 34-68)  Desviacion estandar de cada uno de los 34 atributos anteriores.

### Clases objetivo para predecir:
- anger
- disgust
- happiness
- suprise
- sadness
- fear

In [39]:
emo_df = pd.read_csv('../FeaturesExtraction/dataset_emotion.csv')
emo_df.head()

,Unnamed: 0,ZCR,Energy,Entropy Of Enery,Spectral Centroid,Pitch,Spectral Entropy,Spectral Flux,Spectral Roll-off,MFCC1,...,std26,std27,std28,std29,std30,std31,std32,std33,std34,Emotion
0,0,0.140801,0.034542,2.998446,0.219739,145.967642,1.269595,0.012848,0.238729,-24.386967,...,0.015261,0.033089,0.023998,0.015884,0.020840,0.010203,0.012469,0.005193,0.011702,anger
1,1,0.143055,0.033249,2.991923,0.224251,139.089194,1.372993,0.011736,0.250292,-23.979904,...,0.027955,0.025066,0.026130,0.021838,0.009046,0.015774,0.011017,0.005344,0.012365,anger
2,2,0.132020,0.039794,2.991409,0.209182,125.161082,1.212463,0.013576,0.211296,-24.659123,...,0.017757,0.033223,0.027783,0.017269,0.020207,0.014235,0.010521,0.005646,0.010771,anger
3,3,0.148990,0.029246,3.095875,0.214586,95.731363,1.440230,0.009391,0.247760,-23.407225,...,0.011519,0.028429,0.014736,0.002561,0.006249,0.012041,0.010141,0.013420,0.008775,anger
4,4,0.144013,0.033666,3.123097,0.215850,155.737917,1.357218,0.009991,0.221917,-23.160479,...,0.021216,0.016150,0.019531,0.009489,0.011435,0.008397,0.006290,0.004602,0.007485,anger


## Informacion del dataframe

In [40]:
emo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 70 columns):
Unnamed: 0           72 non-null int64
ZCR                  72 non-null float64
Energy               72 non-null float64
Entropy Of Enery     72 non-null float64
Spectral Centroid    72 non-null float64
Pitch                72 non-null float64
Spectral Entropy     72 non-null float64
Spectral Flux        72 non-null float64
Spectral Roll-off    72 non-null float64
MFCC1                72 non-null float64
MFCC2                72 non-null float64
MFCC3                72 non-null float64
MFCC4                72 non-null float64
MFCC5                72 non-null float64
MFCC6                72 non-null float64
MFCC7                72 non-null float64
MFCC8                72 non-null float64
MFCC9                72 non-null float64
MFCC10               72 non-null float64
MFCC11               72 non-null float64
MFCC12               72 non-null float64
MFCC13               72 non-null float

## Estadisticas del dataframe

In [41]:
emo_df.iloc[0:,1:69].describe()

,ZCR,Energy,Entropy Of Enery,Spectral Centroid,Pitch,Spectral Entropy,Spectral Flux,Spectral Roll-off,MFCC1,MFCC2,...,std25,std26,std27,std28,std29,std30,std31,std32,std33,std34
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,0.128884,0.041371,3.032959,0.227062,121.173553,1.082317,0.013225,0.209568,-25.974959,1.411717,...,0.024533,0.022337,0.023095,0.031057,0.009654,0.015103,0.016141,0.020031,0.011807,0.012951
std,0.027378,0.026066,0.081687,0.028091,34.841668,0.245646,0.004472,0.053355,2.447681,0.579826,...,0.017252,0.011260,0.011267,0.018543,0.006803,0.008084,0.006889,0.009548,0.007804,0.002952
min,0.075313,0.001962,2.719180,0.165945,63.057092,0.551132,0.007279,0.095914,-32.585694,0.189669,...,0.002943,0.005125,0.005612,0.005746,0.001637,0.003105,0.003363,0.003749,0.001290,0.006121
25%,0.110508,0.023884,2.991900,0.210356,98.585313,0.951326,0.010690,0.176141,-27.153698,0.991792,...,0.010850,0.012995,0.013680,0.019242,0.004062,0.008373,0.011450,0.012101,0.006071,0.011330
50%,0.126534,0.037128,3.031428,0.224596,111.615145,1.103044,0.012847,0.209284,-25.526687,1.414974,...,0.021200,0.021091,0.023091,0.026187,0.008365,0.014109,0.014587,0.018875,0.009087,0.012844
75%,0.144206,0.053501,3.075124,0.244760,139.083907,1.238003,0.014331,0.245243,-24.348523,1.876390,...,0.034029,0.028626,0.030857,0.040446,0.013148,0.021376,0.020206,0.025527,0.014933,0.014071
max,0.202983,0.127113,3.200351,0.284338,215.937222,1.819280,0.034300,0.371083,-22.570052,2.440473,...,0.087338,0.054139,0.054628,0.082119,0.036587,0.035333,0.032825,0.047541,0.040294,0.020213


## Mapeamos la ultima columna con valores enteros

In [42]:
mapping = {'anger':0, 'sadness':1,'disgust':2,'fear':3,'hapiness':4,'surprice':5}
emo_df['Emotion'] = emo_df['Emotion'].map(mapping)

## Separacion del conjunto de datos

In [43]:

X = emo_df.iloc[0:,1:69].values #Obtenemos los valores correspondientes a una columna particular
y = emo_df.Emotion                  # Etiquetas correspondientes a cada columna

X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(
    X, y, random_state=0)

In [44]:
print("Forma de X_entrenamiento: {}".format(X_entrenamiento.shape))
print("Forma de y_entrenamiento: {}".format(y_entrenamiento.shape))

Forma de X_entrenamiento: (54, 68)
Forma de y_entrenamiento: (54,)


In [45]:
print("Forma de X_prueba: {}".format(X_prueba.shape))
print("Forma de y_prueba: {}".format(y_prueba.shape))

Forma de X_prueba: (18, 68)
Forma de y_prueba: (18,)


## Seleccion de los Hiper-Parametros

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report



# Fijar los parametros para la validación cruzada
parameters = [{'kernel': ['rbf'],
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
                'C': [1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
             {'kernel': ['poly'],
              'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5],
             'degree': [3, 5, 7],
             'C': [1, 10, 100, 1000]}]

print("# Selección de los hiperparámetros")
print()

clf = GridSearchCV(svm.SVC(decision_function_shape='ovr'), parameters, cv=2)
clf.fit(X_entrenamiento, y_entrenamiento)

print("Mejores parametros encontrados para el conjunto de validación:")
print()
print(clf.best_params_)
print()
print("Valores de exactitud sobre el conjunto de entrenamineto:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

# Selección de los hiperparámetros



/home/neriomoran/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


Mejores parametros encontrados para el conjunto de validación:

{'C': 1, 'kernel': 'linear'}

Valores de exactitud sobre el conjunto de entrenamineto:

0.296 (+/-0.118) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.389 (+/-0.021) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.333 (+/-0.050) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 1, 'gamma': 0.2, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}
0.352 (+/-0.015) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.333 (+/-0.099) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.333 (+/-0.050) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 10, 'gamma': 0.2, 'kernel': 'rbf'}
0.241 (+/-0.001) for {'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
0.315 (+/-0.232) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.315 (+/-0.065) for {'C': 100, 

## Construccion del Modelo


In [47]:
# Create the SVC model object
C = 1.0 # SVM regularization parameter
svc = svm.SVC(kernel='rbf', C=C, decision_function_shape='ovr').fit(X_entrenamiento, y_entrenamiento)
svc.score(X_prueba,y_prueba)



0.3888888888888889

In [48]:
C = 1.0 # SVM regularization parameter
svc = svm.SVC(kernel='poly', C=C, decision_function_shape='ovr').fit(X_entrenamiento, y_entrenamiento)
svc.score(X_prueba,y_prueba)


0.44444444444444442

In [49]:
C = 1.0 # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C, decision_function_shape='ovr').fit(X_entrenamiento, y_entrenamiento)
svc.score(X_prueba,y_prueba)

0.55555555555555558

## Haciendo predicciones

## Matriz de confuncion